In [2]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import ast

## City Bikes Data

In [11]:
# Get all networks from CSV file
city_bikes = pd.read_csv('../data/city_bikes.csv')
# Print the coordinates for the networks
for i, station in city_bikes.head(1).iterrows():
    print('latitude :', station['latitude'], 'longitude :', station['longitude'])

latitude : 52.210833 longitude : 21.007851


In [4]:
# Load environment
load_dotenv()

False

# Foursquare

In [12]:
fsq_api_key = os.getenv('FOURSQUARE_API_KEY')
# Set FourSquare URL
fsq_url = 'https://api.foursquare.com/v3/places/search'
# Create dictionary for headers and add API KEY
fsq_headers = {'Accept': 'application/json', 'Authorization': fsq_api_key}


### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [13]:
# Getting parameters for the api query string
def get_fsq_params(lat, lon):
    return {
        'll': f'{lat},{lon}',
        'radius': 1000,
        'categories': ','.join([
            '4d4b7105d754a06374d81259',  # Restaurant,
            '4bf58dd8d48988d116941735',  # Bar,
            '4bf58dd8d48988d1fa931735'  # Hotel
        ]),
        # 'limit': 10 # max 50
        'fields': 'categories,distance,rating,price'
    }

In [14]:
# Send API requests
def get_fsq_response(url, headers, params):
    return requests.get(url, headers=headers, params=params)

### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [480]:
#FourSquare DataFrame
fsq_df = pd.DataFrame()
for i, station in city_bikes.head(1).iterrows():
    # Set params
    latitude, longitude = station['latitude'], station['longitude']
    fsq_params = get_fsq_params(latitude, longitude)
    # Get response
    fsq_response = get_fsq_response(fsq_url, fsq_headers, fsq_params)
    # Normalize response JSON to fit DataFrame
    df = pd.json_normalize(
        fsq_response.json(), record_path='results'
    )
    fsq_df = pd.concat([fsq_df, df], ignore_index=True)
# Write the data to CSV - IDEALLY SHOULD APPEND  
fsq_df.to_csv('../data/fsq_search_data.csv', sep=',', index=False)

### Put your parsed results into a DataFrame

In [465]:
fsq_df = pd.read_csv('../data/fsq_search_data.csv')
fsq_df.head()

,categories,distance,price,rating
0,"[{'id': 13003, 'name': 'Bar', 'short_name': 'Bar', 'plural_name': 'Bars', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}}, {'id': 13065, 'name': 'Restaurant', 'short_name': 'Restaurant', 'plural_name': 'Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}]",569,2,8.8
1,"[{'id': 13035, 'name': 'Coffee Shop', 'short_name': 'Coffee Shop', 'plural_name': 'Coffee Shops', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}}, {'id': 13046, 'name': 'Ice Cream Parlor', 'short_name': 'Ice Cream', 'plural_name': 'Ice Cream Parlors', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_', 'suffix': '.png'}}, {'id': 13065, 'name': 'Restaurant', 'short_name': 'Restaurant', 'plural_name': 'Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}]",665,2,9.1
2,"[{'id': 13064, 'name': 'Pizzeria', 'short_name': 'Pizza', 'plural_name': 'Pizzerias', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_', 'suffix': '.png'}}, {'id': 13236, 'name': 'Italian Restaurant', 'short_name': 'Italian', 'plural_name': 'Italian Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}}]",765,2,9.0
3,"[{'id': 13003, 'name': 'Bar', 'short_name': 'Bar', 'plural_name': 'Bars', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}}]",847,2,8.8
4,"[{'id': 13276, 'name': 'Sushi Restaurant', 'short_name': 'Sushi', 'plural_name': 'Sushi Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_', 'suffix': '.png'}}]",913,3,8.8


# Yelp

In [15]:
# Get Yelp API Key
yelp_api_key = os.environ['YELP_API_KEY']
# Set Yelp URL
yelp_url = 'https://api.yelp.com/v3/businesses/search'
# Create dictionary for headers and add API KEY
yelp_headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {yelp_api_key}'
}

### Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [16]:
# Getting parameters for the api query string
def get_yelp_params(lat, lon):
    return {
        'latitude': lat,
        'longitude': lon,
        'radius': 1000,
        'limit': 10,
        'categories': 'Bars,Restaurants,Hotels'
    }

In [17]:
# Send API requests
def get_yelp_response(url, headers, params):
    return requests.get(url, headers=headers, params=params)

### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
#Yelp DataFrame
yelp_df = pd.DataFrame()

for i, station in city_bikes.head(1).iterrows():
    # Set params
    latitude, longitude = station['latitude'], station['longitude']
    yelp_params = get_yelp_params(latitude, longitude)
    # Get response
    yelp_response = get_yelp_response(yelp_url, yelp_headers, yelp_params)
    # Normalize response JSON to fit DataFrame
    df = pd.json_normalize(
        yelp_response.json(), record_path='businesses'
    )
    yelp_df = pd.concat([yelp_df, df], ignore_index=True)

# Write the data to CSV - IDEALLY SHOULD APPEND
yelp_df.to_csv('../data/yelp_search_data.csv', sep=',', index=False)

### Put your parsed results into a DataFrame

## FourSquare Data 

In [35]:
#Read from CSV
fsq_df = pd.read_csv('../data/fsq_search_data.csv')
# Safely evaluate the string as a Python dictionary
fsq_df['categories'] = fsq_df['categories'].apply(ast.literal_eval)
# Explode the data by categories
fsq_df = fsq_df.explode('categories')
# Explode the dictionary
categories = fsq_df['categories'].apply(pd.Series)
# Rename the categories columns
categories.columns = ['category.id', 'category.short', 'category.name', 'category.plural', 'category.img_url']
# categories.head()
# Replace categories column by columns
fsq_df = pd.concat([fsq_df.drop(columns=['categories']), categories], axis=1)
# print(fsq_df.head(1))
# Choose columns for EDA
fsq_df = fsq_df[['price', 'rating', 'distance', 'category.plural', 'category.name']]
fsq_df.head()

,price,rating,distance,category.plural,category.name
0,2.0,8.8,569,Bars,Bar
0,2.0,8.8,569,Restaurants,Restaurant
1,2.0,9.1,665,Coffee Shops,Coffee Shop
1,2.0,9.1,665,Ice Cream Parlors,Ice Cream
1,2.0,9.1,665,Restaurants,Restaurant


## YELP Data

In [22]:
#Read from CSV
yelp_df = pd.read_csv('../data/yelp_search_data.csv')
# Safely evaluate the string as a Python dictionary
yelp_df['categories'] = yelp_df['categories'].apply(ast.literal_eval)
# Explode the data by categories
yelp_df = yelp_df.explode('categories')
# Explode the dictionary
categories = yelp_df['categories'].apply(pd.Series)
# Rename the categories columns
categories.columns = ['category.alias', 'category.title']
# Replace categories column by columns
yelp_df = pd.concat([yelp_df.drop(columns=['categories']), categories], axis=1)
# Choose columns for EDA
yelp_df = yelp_df[['name', 'review_count', 'rating', 'price', 'distance', 'business_hours', 'category.title']]
yelp_df.head()

,name,review_count,rating,price,distance,business_hours,category.title
0,Pub Zielona Gęś,14,3.3,$$,123.908924,"[{'open': [{'is_overnight': True, 'start': '09...",Sports Bars
1,Ministerstwo Kawy,51,4.2,$,1109.084599,"[{'open': [{'is_overnight': False, 'start': '0...",Cafes
1,Ministerstwo Kawy,51,4.2,$,1109.084599,"[{'open': [{'is_overnight': False, 'start': '0...",Coffee & Tea
2,Pardon To Tu,28,4.4,$$,850.986161,"[{'open': [{'is_overnight': False, 'start': '0...",Vinyl Records
2,Pardon To Tu,28,4.4,$$,850.986161,"[{'open': [{'is_overnight': False, 'start': '0...",Bookstores


# Comparing Results

### Which API provided you with more complete data? Provide an explanation? 

#### Each APIs have different approach to handle the data. The outcome of a query depends on various parameters. I wish I could test all the possible options

### Get the top 10 restaurants according to their rating

### FourSquare Top 10 by ranking 

In [36]:
# Remove duplicate rows
df = fsq_df.drop_duplicates().copy()
# Sort data by rating
df.sort_values(by='rating', ascending=False, inplace=True)
# Display top 10
df.head(10)

,price,rating,distance,category.plural,category.name
2162,2.0,9.3,982,Bars,Bar
492,2.0,9.3,874,Bars,Bar
1101,2.0,9.3,961,Bars,Bar
2651,2.0,9.3,764,Bars,Bar
1921,2.0,9.3,941,Bars,Bar
3068,2.0,9.3,369,Bars,Bar
608,2.0,9.3,107,Bars,Bar
3020,2.0,9.3,428,Bars,Bar
3000,2.0,9.3,671,Bars,Bar
1941,2.0,9.3,582,Bars,Bar


### YELP Top 10 by ranking

In [37]:
# Remove duplicate rows
df = yelp_df.drop_duplicates().copy()
# Sort data by rating
df.sort_values(by='rating', ascending=False, inplace=True)
# Display top 10
df.head(10)

,name,review_count,rating,price,distance,business_hours,category.title
3,Ciao a Tutti,32,4.7,$$,771.374846,"[{'open': [{'is_overnight': False, 'start': '1...",Italian
3,Ciao a Tutti,32,4.7,$$,771.374846,"[{'open': [{'is_overnight': False, 'start': '1...",Pizza
7,Green Caffè Nero,11,4.6,$$,945.172178,"[{'open': [{'is_overnight': False, 'start': '0...",Cafes
2,Pardon To Tu,28,4.4,$$,850.986161,"[{'open': [{'is_overnight': False, 'start': '0...",Vinyl Records
2,Pardon To Tu,28,4.4,$$,850.986161,"[{'open': [{'is_overnight': False, 'start': '0...",Bookstores
2,Pardon To Tu,28,4.4,$$,850.986161,"[{'open': [{'is_overnight': False, 'start': '0...",Pubs
5,Krowarzywa,20,4.3,$,1124.132272,"[{'open': [{'is_overnight': False, 'start': '1...",Vegan
5,Krowarzywa,20,4.3,$,1124.132272,"[{'open': [{'is_overnight': False, 'start': '1...",Burgers
1,Ministerstwo Kawy,51,4.2,$,1109.084599,"[{'open': [{'is_overnight': False, 'start': '0...",Cafes
1,Ministerstwo Kawy,51,4.2,$,1109.084599,"[{'open': [{'is_overnight': False, 'start': '0...",Coffee & Tea


## Testing Area

In [473]:
# # Connect to SQLite database (or create it if it doesn't exist)
# conn = sqlite3.connect('API.db')
# 
# # Save DataFrame to SQLite table
# df.to_sql('yelp', conn, if_exists='replace', index=False)
# 
# # Close the connection
# conn.close()
# 
# print('Data saved to SQLite database!')

In [474]:
# Open and read the JSON file
# with open('fsq_search_response.json', 'r') as file:
#     data = json.load(file)
# data = pd.read_json('fsq_search_columns.json')
# data = data.explode('categories').reset_index(drop=True)
# data['categories'] = data['categories'].apply(lambda x: x['short_name'])
# data.drop([
#     'geocodes.drop_off.latitude',
#     'geocodes.drop_off.longitude',
#     'geocodes.roof.latitude',
#     'geocodes.roof.longitude',
# ], axis=1, inplace=True)
# data